In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,60,2024-03-02,-1,120.745763,100.566102,0.538712,10.847458,27.659322,0.217169,119.176271,...,10.725424,24.711864,0.174017,115.074576,0,3,5,2,1,2024
1,61,2024-03-02,-2,117.483333,99.431667,0.543217,13.098333,28.725000,0.213267,117.161667,...,11.403390,22.238983,0.221729,114.686441,0,3,5,2,29,2024
2,58,2024-03-02,-9,107.245614,96.705263,0.502772,12.436842,26.524561,0.195789,109.791228,...,12.518644,21.952542,0.181915,108.442373,0,3,5,4,25,2024
3,61,2024-03-02,5,114.383333,96.150000,0.557633,10.823333,25.940000,0.178667,119.100000,...,11.795082,19.555738,0.212279,115.862295,0,3,5,2,8,2024
4,60,2024-03-02,-3,112.694915,98.542373,0.523949,11.194915,24.674576,0.205864,113.442373,...,12.591525,24.793220,0.241627,119.281356,0,3,5,2,11,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-02,ATL,BRK,1.869565,1.952381,0
1,2024-03-02,UTA,MIA,3.350000,1.344828,0
2,2024-03-02,POR,MEM,2.200000,1.704225,0
3,2024-03-02,DEN,LAL,1.909091,1.909091,1
4,2024-03-02,HOU,PHO,4.400000,1.227273,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-02,ATL,BRK,1.869565,1.952381,0
1,2024-03-02,UTA,MIA,3.350000,1.344828,0
2,2024-03-02,POR,MEM,2.200000,1.704225,0
3,2024-03-02,DEN,LAL,1.909091,1.909091,0
4,2024-03-02,HOU,PHO,4.400000,1.227273,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-02,ATL,BRK,1.869565,1.952381,0
1,2024-03-02,UTA,MIA,3.350000,1.344828,0
2,2024-03-02,POR,MEM,2.200000,1.704225,0
3,2024-03-02,DEN,LAL,1.909091,1.909091,0
4,2024-03-02,HOU,PHO,4.400000,1.227273,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-02,ATL,BRK,1.869565,1.952381,0
1,2024-03-02,UTA,MIA,3.350000,1.344828,0
2,2024-03-02,POR,MEM,2.200000,1.704225,0
3,2024-03-02,DEN,LAL,1.909091,1.909091,1
4,2024-03-02,HOU,PHO,4.400000,1.227273,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-02,ATL,BRK,1.869565,1.952381,1.0
1,2024-03-02,UTA,MIA,3.350000,1.344828,1.0
2,2024-03-02,POR,MEM,2.200000,1.704225,0.0
3,2024-03-02,DEN,LAL,1.909091,1.909091,1.0
4,2024-03-02,HOU,PHO,4.400000,1.227273,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-02,ATL,BRK,1.869565,1.952381,1.0
1,2024-03-02,UTA,MIA,3.350000,1.344828,1.0
2,2024-03-02,POR,MEM,2.200000,1.704225,0.0
3,2024-03-02,DEN,LAL,1.909091,1.909091,1.0
4,2024-03-02,HOU,PHO,4.400000,1.227273,0.0
